# Algoritmos de Classificação de Lúpus


## Imports


In [1]:
!pip install tqdm
from tqdm import tqdm

Defaulting to user installation because normal site-packages is not writeable


## Pacientes

In [2]:
columns_aih = ['id_paciente', 'co_cid_principal', 'no_cid_principal', 'co_procedimento_principal',
               'co_cid_secundario', 'no_cid_secundario', 'procedimento_principal', 'desc_procedimento_secundario']

columns_apac = ['id_paciente', 'co_cid_principal', 'no_cid_principal', 'co_cid_secundario',
              'co_procedimento_principal', 'no_procedimento_principal', 'co_procedimento_secundario', 'no_procedimento_secundario']

columns_bpai = ['id_paciente', 'co_cid_principal', 'no_cid_principal', 'co_procedimento_realizado', 'no_procedimento_realizado', ]

In [3]:
import pandas as pd
path = 'data/'
pac_apac = pd.read_parquet(path+'APAC_Lupus_L93M32N08_todos_cids.parquet', columns=columns_apac)
pac_aih = pd.read_parquet(path+'AIH_Lupus_L93M32N08_todos_cids.parquet', columns=columns_aih)
pac_bpai = pd.read_parquet(path+'BPAI_Lupus_L93M32N08_todos_cids.parquet', columns=columns_bpai)

In [4]:
print(f' - APAC: Registros: {pac_apac.shape[0]} Pacientes únicos: {pac_apac['id_paciente'].nunique()}')
print(f' - AIH: Registros: {pac_aih.shape[0]} Pacientes únicos: {pac_aih['id_paciente'].nunique()}')
print(f' - BPAI: Registros: {pac_bpai.shape[0]} Pacientes únicos: {pac_bpai['id_paciente'].nunique()}')
print(f' - Total: Registros: {pac_bpai.shape[0]+pac_aih.shape[0]+pac_apac.shape[0]} Pacientes únicos: {pac_bpai['id_paciente'].nunique()+pac_aih['id_paciente'].nunique()+pac_apac['id_paciente'].nunique()}')


 - APAC: Registros: 7194178 Pacientes únicos: 31948
 - AIH: Registros: 3968946 Pacientes únicos: 48663
 - BPAI: Registros: 857369 Pacientes únicos: 46967
 - Total: Registros: 12020493 Pacientes únicos: 127578


In [5]:
set_bpai = set(pac_bpai['id_paciente'])
set_apac = set(pac_apac['id_paciente'])
set_aih = set(pac_aih['id_paciente'])

print(' - Interseção do aih com apac:', len(set_aih.intersection(set_apac)))
print(' - Interseção do aih com bpai:', len(set_bpai.intersection(set_aih)))
print(' - Interseção do bpai com apac:', len(set_apac.intersection(set_bpai)))
print(' - Interseção dos 3 conjuntos:', len(set_apac.intersection(set_bpai).intersection(set_aih)))

 - Interseção do aih com apac: 28856
 - Interseção do aih com bpai: 25601
 - Interseção do bpai com apac: 19485
 - Interseção dos 3 conjuntos: 18238


## Filtrando CIDs

In [6]:
def verifica_cid(row):
    for cod in ['M32', 'L93', 'N08']:
        if (cod in row):
            return True    
    return False

def filtro_cid(df_paciente, col_cid):
    if isinstance(df_paciente, pd.DataFrame):
        res = df_paciente[col_cid].map(verifica_cid)
        return res.sum() > 0
    else:
        return verifica_cid(df_paciente[col_cid])

# Conjunto com os pacientes que passaram na verificação do cid
set_pacientes_cid = set()

for nome_df, df in zip(['AIH', 'BPAI', 'APAC'],[pac_aih, pac_bpai, pac_apac]):
    print(' - Começando', nome_df)
    if df.index.name != 'id_paciente':
        df.set_index('id_paciente', inplace=True)

    for col in df.filter(regex='co_cid_').columns:
        print(col)

        for id_paciente in tqdm(set(df.index)):
            df_paciente = df.loc[id_paciente]
        
            if filtro_cid(df_paciente, col): # se passou no filtro de cid
                set_pacientes_cid.add(id_paciente) # salva o id_paciente
    df.reset_index(inplace=True)
    #df['filtro_cid_principal'] = df['co_cid_principal'].apply(verifica_cid)
    #if 'co_cid_secundario' in df.columns:
    #    df['filtro_cid_secundario'] = df['co_cid_secundario'].apply(verifica_cid)

 - Começando AIH
co_cid_principal


100%|██████████| 48663/48663 [08:46<00:00, 92.45it/s] 


co_cid_secundario


100%|██████████| 48663/48663 [07:10<00:00, 113.11it/s]


 - Começando BPAI
co_cid_principal


100%|██████████| 46967/46967 [00:57<00:00, 818.75it/s]


 - Começando APAC
co_cid_principal


100%|██████████| 31948/31948 [06:59<00:00, 76.18it/s]


co_cid_secundario


100%|██████████| 31948/31948 [08:15<00:00, 64.46it/s]


In [7]:
len(set_pacientes_cid)

69772

In [8]:
pac_bpai['filtro_cid'] = pac_bpai['id_paciente'].isin(set_pacientes_cid)
pac_aih['filtro_cid'] = pac_aih['id_paciente'].isin(set_pacientes_cid)
pac_apac['filtro_cid'] = pac_apac['id_paciente'].isin(set_pacientes_cid)

In [9]:
def calc_percent_cid(df):
    res = df[df['filtro_cid']]['id_paciente'].nunique()/df['id_paciente'].nunique()
    return round(res*100,2)

print(' - Pacientes do BPAI aprovados no pelo cid:', calc_percent_cid(pac_bpai),'%')
print(' - Pacientes do AIH aprovados no pelo cid:', calc_percent_cid(pac_aih),'%')
print(' - Pacientes do APAC aprovados no pelo cid:', calc_percent_cid(pac_apac),'%')

 - Pacientes do BPAI aprovados no pelo cid: 97.2 %
 - Pacientes do AIH aprovados no pelo cid: 95.68 %
 - Pacientes do APAC aprovados no pelo cid: 96.02 %


## Classificando os Pacientes

In [10]:
import importlib
from  src import procedimentos
importlib.reload(procedimentos)

def verifica_procedimentos(df_pacientes, col_procedimento):
    df_pacientes = df_pacientes.copy()
    pacientes_list = list(df_pacientes.id_paciente.unique())
    dict_pacientes = {}
    proc_dict = {'p1':procedimentos.p1, 'p2':procedimentos.p2, 'p3':procedimentos.p3, 'p4':procedimentos.p4}
    
    for proc_name in proc_dict.keys():
        dict_pacientes[proc_name] = set()

    for id_paciente in tqdm(pacientes_list, desc=f'Classificando Pacientes'):
        id_paciente = int(id_paciente)
        df_paciente = df_pacientes[df_pacientes.id_paciente==id_paciente].copy() # Pega apenas os procedimentos no paciente em questão        

        for proc, proc_func in proc_dict.items():
            if proc_func(df_paciente, col_procedimento): # Aplica todos os procedimentos em cada cliente
                dict_pacientes[proc].add(id_paciente) # Salva o id do paciente no conjunto do procedimento em que ele passou

    for proced in ['p1', 'p2', 'p3', 'p4']: # Adiciona uma coluna como True para cada procedimento em que o paciente foi aprovado
        df_pacientes[f'procedimento_{proced}'] = df_pacientes['id_paciente'].isin(dict_pacientes[proced])
    
    return df_pacientes

In [14]:
df_aih = verifica_procedimentos(pac_aih, col_procedimento='desc_procedimento_secundario')
df_bpai = verifica_procedimentos(pac_bpai, col_procedimento='no_procedimento_realizado')
df_apac = verifica_procedimentos(pac_apac, col_procedimento='no_procedimento_principal')

Classificando Pacientes: 100%|██████████| 31948/31948 [06:34<00:00, 81.03it/s] 


In [15]:
# apenas para garantir, apagar depois
#df_aih['filtro_cid'] = pac_aih['filtro_cid']
#df_bpai['filtro_cid'] = pac_bpai['filtro_cid']
#df_apac['filtro_cid'] = pac_apac['filtro_cid']

### Aplicando Filtros
Os filtros são aplicados antes dos algoritmos para diminuir o tamanho do conjunto

In [16]:
# Juntando as bases (BPAI, APAC e AIH)
cols_union = ['id_paciente', 'procedimento_p1', 'procedimento_p2',
              'procedimento_p3', 'procedimento_p4', 'filtro_cid']
df_union = pd.concat([df_bpai[cols_union],
                      df_apac[cols_union],
                      df_aih[cols_union]])
df_union.fillna(False, inplace=True)

In [17]:
print('Número de registros')
print(f' - AIH: {df_aih.shape[0]}')
print(f' - BPAI: {df_bpai.shape[0]}')
print(f' - APAC: {df_apac.shape[0]}')
print()
print('Número de Pacientes Únicos:')
print(f' - AIH: {df_aih['id_paciente'].nunique()}')
print(f' - BPAI: {df_bpai['id_paciente'].nunique()}')
print(f' - APAC: {df_apac['id_paciente'].nunique()}')

Número de registros
 - AIH: 3968946
 - BPAI: 857369
 - APAC: 7194178

Número de Pacientes Únicos:
 - AIH: 48663
 - BPAI: 46967
 - APAC: 31948


In [18]:
print(' - Tamanho da união dos conjuntos:', df_union.shape)
print(' - Pacientes totais:', df_union['id_paciente'].nunique())

 - Tamanho da união dos conjuntos: (12020493, 6)
 - Pacientes totais: 71874


In [19]:
df_cid = df_union[df_union['filtro_cid']].copy()
print(df_cid.shape)
print(df_cid['id_paciente'].nunique())

(11647918, 6)
69772


In [20]:
df_cid = df_union[df_union['filtro_cid']].copy()
print('Número de Registros:')
print(' - Aplicando o filtro do cid')
print(f' - Redução de {df_union.shape[0]} para {df_cid.shape[0]} = {round((df_union.shape[0]-df_cid.shape[0])*100/df_union.shape[0],2)}%')

print()
num_pac_union = df_union['id_paciente'].nunique()
num_pac_cid = df_cid['id_paciente'].nunique()
print('Número de Pacientes Únicos:')
print(f' - Antes do filtro: {num_pac_union}')
print(f' - Após o filtro: {num_pac_cid}')
red_pac = (num_pac_union-num_pac_cid)/num_pac_union
print(f' - Redução de {round(red_pac*100,2)}%')
print(f' - Ou seja, {round(num_pac_cid*100/num_pac_union,2)}% aprovados no filtro do cid')

Número de Registros:
 - Aplicando o filtro do cid
 - Redução de 12020493 para 11647918 = 3.1%

Número de Pacientes Únicos:
 - Antes do filtro: 71874
 - Após o filtro: 69772
 - Redução de 2.92%
 - Ou seja, 97.08% aprovados no filtro do cid


In [21]:
# Agora aplicando os filtros de procedimento
cols_proced = ['procedimento_p1', 'procedimento_p2', 'procedimento_p3', 'procedimento_p4']
df_cid_pac = df_cid[df_cid['procedimento_p1'] | df_cid['procedimento_p2'] | 
                    df_cid['procedimento_p3'] | df_cid['procedimento_p4']].groupby('id_paciente')[cols_proced].sum()

df_cid_filtro_proced = df_cid_pac.map(lambda row: 1 if row > 0 else 0)
df_final = df_cid_filtro_proced.sum(axis=1).sort_values()

#### Quantos pacientes passaram em cada procedimento

In [35]:
df_cid_filtro_proced.sum()

procedimento_p1       11
procedimento_p2    26974
procedimento_p3    10630
procedimento_p4    14998
dtype: int64

### Algoritmo 1
CID + 2 Procedimentos distindos

In [23]:
# Atualizado
df_alg1 = df_final[df_final > 1].to_frame().reset_index()
df_alg1.columns = ['id_paciente', 'num_procedimentos_qualificados']
print(' - Total de pacientes encontrados após o algoritmo 1:', df_alg1.shape[0])
print(f' - % em relação ao total de pacientes (BPAI+APAC+AIH): \
{round((df_alg1.shape[0]*100)/df_union['id_paciente'].nunique(),2)}%')

 - Total de pacientes encontrados após o algoritmo 1: 15445
 - % em relação ao total de pacientes (BPAI+APAC+AIH): 21.49%


### Algoritmo 2
CID + 1 Procedimento

In [25]:
# Atualizado
df_alg2 = df_final[df_final > 0].to_frame().reset_index()
df_alg2.columns = ['id_paciente', 'num_procedimentos_qualificados']
print(' - Total de pacientes encontrados após o algoritmo 2:', df_alg2.shape[0])
print(f' - % em relação ao total de pacientes (BPAI+APAC+AIH): \
{round((df_alg2.shape[0]*100)/df_union['id_paciente'].nunique(),2)}%')

 - Total de pacientes encontrados após o algoritmo 2: 31583
 - % em relação ao total de pacientes (BPAI+APAC+AIH): 43.94%


### Salvando os Resultados

In [26]:
df_alg1.to_parquet('data/results/df_alg1.parquet')
df_alg2.to_parquet('data/results/df_alg2.parquet')

In [27]:
df_bpai_alg1 = df_bpai[df_bpai['id_paciente'].isin(df_alg1['id_paciente'])]
df_aih_alg1 = df_aih[df_aih['id_paciente'].isin(df_alg1['id_paciente'])]
df_apac_alg1 = df_apac[df_apac['id_paciente'].isin(df_alg1['id_paciente'])]

df_bpai_alg1.to_parquet('data/results/df_bpai_alg1.parquet')
df_aih_alg1.to_parquet('data/results/df_aih_alg1.parquet')
df_apac_alg1.to_parquet('data/results/df_apac_alg1.parquet')

In [28]:
df_bpai_alg2 = df_bpai[df_bpai['id_paciente'].isin(df_alg2['id_paciente'])]
df_aih_alg2 = df_aih[df_aih['id_paciente'].isin(df_alg2['id_paciente'])]
df_apac_alg2 = df_apac[df_apac['id_paciente'].isin(df_alg2['id_paciente'])]

df_bpai_alg2.to_parquet('data/results/df_bpai_alg2.parquet')
df_aih_alg2.to_parquet('data/results/df_aih_alg2.parquet')
df_apac_alg2.to_parquet('data/results/df_apac_alg2.parquet')

## Visão Geral dos Resultados

### Algoritmo 1

In [29]:
print(' - Registros dos pacientes aprovados no Algoritmo 1')
print('- BPAI:', df_bpai_alg1.shape)
print(' - AIH:', df_aih_alg1.shape)
print(' - APAC:', df_apac_alg1.shape)

 - Registros dos pacientes aprovados no Algoritmo 1
- BPAI: (356035, 10)
 - AIH: (2069241, 13)
 - APAC: (1281196, 13)


In [30]:
total = df_alg1['id_paciente'].nunique()
print(' - Pacientes presentes no bpai:', round(df_bpai_alg1['id_paciente'].nunique() / total, 2))
print(' - Pacientes presentes no aih:', round(df_aih_alg1['id_paciente'].nunique() / total, 2))
print(' - Pacientes presentes no apac:', round(df_apac_alg1['id_paciente'].nunique() / total, 2))

 - Pacientes presentes no bpai: 0.69
 - Pacientes presentes no aih: 0.97
 - Pacientes presentes no apac: 0.8


### Algoritmo 2

In [31]:
print(' - Registros dos pacientes aprovados no Algoritmo 2')
print('- BPAI:', df_bpai_alg2.shape)
print(' - AIH:', df_aih_alg2.shape)
print(' - APAC:', df_apac_alg2.shape)

 - Registros dos pacientes aprovados no Algoritmo 2
- BPAI: (520997, 10)
 - AIH: (3298356, 13)
 - APAC: (3676103, 13)


In [32]:
total = df_alg2['id_paciente'].nunique()
print(' - Pacientes presentes no bpai:', round(df_bpai_alg2['id_paciente'].nunique() / total, 2))
print(' - Pacientes presentes no aih:', round(df_aih_alg2['id_paciente'].nunique() / total, 2))
print(' - Pacientes presentes no apac:', round(df_apac_alg2['id_paciente'].nunique() / total, 2))

 - Pacientes presentes no bpai:

 0.61
 - Pacientes presentes no aih: 0.96
 - Pacientes presentes no apac: 0.69


In [33]:
# Atualizado
total = df_alg2['id_paciente'].nunique()
print(' - Pacientes presentes no bpai:', round(df_bpai_alg2['id_paciente'].nunique() / total, 2))
print(' - Pacientes presentes no aih:', round(df_aih_alg2['id_paciente'].nunique() / total, 2))
print(' - Pacientes presentes no apac:', round(df_apac_alg2['id_paciente'].nunique() / total, 2))

 - Pacientes presentes no bpai: 0.61
 - Pacientes presentes no aih: 0.96
 - Pacientes presentes no apac: 0.69
